In [ ]:
import meep as mp
import numpy as np
from matplotlib import pyplot as plt
from meep.materials import Si
from meep import mpb


resolution = 10

n_meep = []
n_mpb = []

My_LiNbO3= mp.Medium(epsilon_diag=mp.Vector3(5.1092, 4.7515, 5.1092)) # X cut
fcen= 1.0/ 0.775 # 775 nm
n = np.sqrt(My_LiNbO3.epsilon(fcen)[0,0])
freq = 1/6


Lpml= 20.0/ resolution

Sx= 2.0# 높이: 위아래 pml 0부터 LN, 그아래는 SiO2
Sy= 3.0# 너비: 좌우 pml, 폭 3um                         -|-
Sz= 3.0# 길이 (진행방향): 3 

Air= mp.Medium(index=1.0)
SiO2= mp.Medium(index=1.52)#<-------------------------------------------------------------경우에 따라 제거할수도
My_LiNbO3= mp.Medium(epsilon_diag=mp.Vector3(5.1092, 4.7515, 5.1092)) # X cut
# pml 두께 여기서 들어감.
cell= mp.Vector3(Sx+ 2*Lpml, Sy+ 2*Lpml, Sz+ 2*Lpml)

# 공간 비례 변수 (pml 뺀 유효 공간)
X_min= 0.5*-Sx
Y_min= 0.5*-Sy
Z_min= 0.5*-Sz

X_max= 0.5*Sx
Y_max= 0.5*Sy
Z_max= 0.5*Sz

# 높이 (x: thickness)
LNsub_h= 0.1 # LN substrate
SiO2_h= 1.0 # SiO2 substrate
LNwg_h= 0.5 # LN waveguide

# 폭 (y: width)
min_g= 0.424 # minimum gap
w_top= 0.798 # top width (output waveguide)
w_bot= w_top+ min_g #bottom width (1222 nm) 

Target_size = mp.Vector3(LNsub_h + LNwg_h, 1.4, 0)


# 구조물 
geometry= [
    mp.Block(#SiO2 substrate
        center= mp.Vector3(X_min+ 0.5*SiO2_h, 0, 0+ Lpml/2),
        material= SiO2,
        size= mp.Vector3(SiO2_h, Sy, Sz+ Lpml)
    ),
    mp.Block(#LiNbO3 substrate
        center= mp.Vector3(X_min+ SiO2_h+ 0.5*LNsub_h, 0, 0+ Lpml/2),
        material= My_LiNbO3,
        size= mp.Vector3(LNsub_h, Sy, Sz+ Lpml)
    ),    
    mp.Prism(#LiNbO3 waveguide
        vertices = [
            mp.Vector3(X_min+ SiO2_h+ LNsub_h, w_bot/2, Z_min),
            mp.Vector3(X_min+ SiO2_h+ LNsub_h, -w_bot/2, Z_min),
            mp.Vector3(X_min+ SiO2_h+ LNsub_h+ LNwg_h, -w_top/2, Z_min),
            mp.Vector3(X_min+ SiO2_h+ LNsub_h+ LNwg_h, w_top/2, Z_min)
        ],
        height= Sz+Lpml,
        axis= mp.Vector3(0, 0, 1),
        sidewall_angle= 0,
        material= My_LiNbO3,
    )
]

pml_layers = [mp.PML(thickness=Lpml)]#, direction=mp.Y)]

res = np.linspace(1,50,10)
for resolution in res:

    # Meep simulation that uses MPB
    sim = mp.Simulation(
        cell_size= cell,
        resolution= resolution,
        boundary_layers= pml_layers,
        eps_averaging= False,
        geometry= geometry,
    )

    sim.init_sim()
    where = mp.Volume(center=mp.Vector3(), size= mp.Vector3(LNsub_h + LNwg_h, 1.4, 0))
    direction = mp.Z
    band_num = 3
    kpoint = mp.Vector3(0,0,1)
    ed = sim.get_eigenmode(fcen, direction, where, band_num, kpoint, eig_vol=None, match_frequency=True, 
    parity=mp.NO_PARITY, resolution=0, eigensolver_tol=1e-16)

    k = ed.k
    n_meep.append(k.z/(fcen))

#     # Direct MPB simulation
#     geometry_lattice = mp.Lattice(size=mp.Vector3(0,2,2))
#     ms = mpb.ModeSolver(
#         geometry_lattice=geometry_lattice,
#         resolution=resolution,
#         num_bands=1,
#         ensure_periodicity=False,
#         default_material=mp.Medium(index=n)
#     )

#     k_mpb = ms.find_k(mp.NO_PARITY, freq, 1, 1, mp.Vector3(1), 1e-3, freq * n, freq * 0.1, freq * 4)
#     n_mpb.append(k_mpb[0]/freq)

plt.figure()
plt.hlines(n,min(res),max(res),label='Si',color='blue',linewidth=2)
plt.plot(res,n_meep,'-o',label='MEEP',color='red',linewidth=2)
# plt.plot(res,n_mpb,'--o',label='MPB',color='green',linewidth=2)
plt.legend()
plt.grid(True)
plt.ylabel('Index of Si at {} $\mu$m'.format(1/freq))
plt.xlabel('Resolution')
plt.tight_layout()
plt.savefig('meep_mpb_comparison.png')
plt.show()

-----------
Initializing structure...
time for choose_chunkdivision = 0.000141144 s
Working in 3D dimensions.
Computational cell is 6 x 7 x 7 with resolution 1
     block, center = (-0.5,0,1)
          size (1,3,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2.3104,2.3104,2.3104)
     block, center = (0.05,0,1)
          size (0.1,3,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (5.1092,4.7515,5.1092)
     prism, center = (0.35,0,1)
          height 5, axis (0,0,1), sidewall angle: 0 radians, 4 vertices:
          (0.1,0.611,-1.5)
          (0.1,-0.611,-1.5)
          (0.6,-0.399,-1.5)
          (0.6,0.399,-1.5)
          dielectric constant epsilon diagonal = (5.1092,4.7515,5.1092)
time for set_epsilon = 0.000283003 s
-----------


MPB solved for frequency_2(0,0,1) = 0.6429 after 40 iters
MPB solved for frequency_2(0,0,2.50171) = 1.36778 after 5 iters
MPB solved for frequency_2(0,0,2.34891) = 1.29051 after 1 iters
MPB solved for frequency_2(0,0,2.34855) = 1.29032 after 1 iters
MPB solved for frequency_2(0,0,2.34855) = 1.29032 after 4 iters
Dominant planewave for band 2: (0.000000,0.714286,2.348551)


-----------
Initializing structure...
time for choose_chunkdivision = 0.000234127 s
Working in 3D dimensions.
Computational cell is 6.05172 x 6.98276 x 6.98276 with resolution 6.44444
     block, center = (-0.5,0,1)
          size (1,3,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2.3104,2.3104,2.3104)
     block, center = (0.05,0,1)
          size (0.1,3,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (5.1092,4.7515,5.1092)
     prism, center = (0.35,0,1)
          height 5, axis (0,0,1), sidewall angle: 0 radians, 4 vertices:
          (0.1,0.611,-1.5)
          (0.1,-0.611,-1.5)
          (0.6,-0.399,-1.5)
          (0.6,0.399,-1.5)
          dielectric constant epsilon diagonal = (5.1092,4.7515,5.1092)
time for set_epsilon = 0.0281241 s
-----------


MPB solved for frequency_2(0,0,1) = 0.574775 after 24 iters
MPB solved for frequency_2(0,0,2.32369) = 1.20626 after 36 iters
MPB solved for frequency_2(0,0,2.51736) = 1.28984 after 40 iters
MPB solved for frequency_2(0,0,2.51848) = 1.29032 after 21 iters
MPB solved for frequency_2(0,0,2.51848) = 1.29032 after 1 iters
Dominant planewave for band 2: (0.000000,0.000000,2.518485)


-----------
Initializing structure...
time for choose_chunkdivision = 0.000305891 s
Working in 3D dimensions.
Computational cell is 5.97196 x 6.98131 x 6.98131 with resolution 11.8889
     block, center = (-0.5,0,1)
          size (1,3,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2.3104,2.3104,2.3104)
     block, center = (0.05,0,1)
          size (0.1,3,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (5.1092,4.7515,5.1092)
     prism, center = (0.35,0,1)
          height 5, axis (0,0,1), sidewall angle: 0 radians, 4 vertices:
          (0.1,0.611,-1.5)
          (0.1,-0.611,-1.5)
          (0.6,-0.399,-1.5)
          (0.6,0.399,-1.5)
          dielectric constant epsilon diagonal = (5.1092,4.7515,5.1092)
time for set_epsilon = 0.159253 s
-----------


...

-----------
Initializing structure...
time for choose_chunkdivision = 0.00082016 s
Working in 3D dimensions.
Computational cell is 6 x 7.0099 x 7.0099 with resolution 33.6667
     block, center = (-0.5,0,1)
          size (1,3,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2.3104,2.3104,2.3104)
     block, center = (0.05,0,1)
          size (0.1,3,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (5.1092,4.7515,5.1092)
     prism, center = (0.35,0,1)
          height 5, axis (0,0,1), sidewall angle: 0 radians, 4 vertices:
          (0.1,0.611,-1.5)
          (0.1,-0.611,-1.5)
          (0.6,-0.399,-1.5)
          (0.6,0.399,-1.5)
          dielectric constant epsilon diagonal = (5.1092,4.7515,5.1092)
time for set_epsilon = 3.40783 s
-----------


MPB solved for frequency_2(0,0,1) = 0.5644 after 24 iters
MPB solved for frequency_2(0,0,2.37846) = 1.20511 after 50 iters
MPB solved for frequency_2(0,0,2.5784) = 1.28997 after 30 iters
MPB solved for frequency_2(0,0,2.57922) = 1.29032 after 16 iters
MPB solved for frequency_2(0,0,2.57922) = 1.29032 after 3 iters
Dominant planewave for band 2: (0.000000,0.000000,2.579219)


-----------
Initializing structure...
time for choose_chunkdivision = 0.00101686 s
Working in 3D dimensions.
Computational cell is 6.00852 x 7.00568 x 7.00568 with resolution 39.1111
     block, center = (-0.5,0,1)
          size (1,3,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2.3104,2.3104,2.3104)
     block, center = (0.05,0,1)
          size (0.1,3,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (5.1092,4.7515,5.1092)
     prism, center = (0.35,0,1)
          height 5, axis (0,0,1), sidewall angle: 0 radians, 4 vertices:
          (0.1,0.611,-1.5)
          (0.1,-0.611,-1.5)
          (0.6,-0.399,-1.5)
          (0.6,0.399,-1.5)
          dielectric constant epsilon diagonal = (5.1092,4.7515,5.1092)


MPB solved for frequency_2(0,0,1) = 0.563502 after 24 iters
MPB solved for frequency_2(0,0,2.38059) = 1.20535 after 32 iters
MPB solved for frequency_2(0,0,2.5799) = 1.28998 after 30 iters
MPB solved for frequency_2(0,0,2.58072) = 1.29032 after 51 iters
MPB solved for frequency_2(0,0,2.58072) = 1.29032 after 1 iters
MPB solved for frequency_2(0,0,2.58072) = 1.29032 after 3 iters
Dominant planewave for band 2: (0.000000,0.000000,2.580720)


-----------
Initializing structure...
time for choose_chunkdivision = 0.00100207 s
Working in 3D dimensions.
Computational cell is 5.99252 x 7.00249 x 7.00249 with resolution 44.5556
     block, center = (-0.5,0,1)
          size (1,3,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2.3104,2.3104,2.3104)
     block, center = (0.05,0,1)
          size (0.1,3,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (5.1092,4.7515,5.1092)
     prism, center = (0.35,0,1)
          height 5, axis (0,0,1), sidewall angle: 0 radians, 4 vertices:
          (0.1,0.611,-1.5)
          (0.1,-0.611,-1.5)
          (0.6,-0.399,-1.5)
          (0.6,0.399,-1.5)
          dielectric constant epsilon diagonal = (5.1092,4.7515,5.1092)
